In [ ]:
# qiime2 activation

In [ ]:
source activate qiime2-2020.6

In [ ]:
##############
# for bacteria
cd ./16S_ONT_1400_1600_q10/

qiime tools import \
--input-path ./seqtable_mangrove_bact_ONT.fa \
--type 'FeatureData[Sequence]' \
--output-path ./rep-seqs_mangrove_bact_ONT.qza

##############
# for archaea
cd ./arch_ONT_900_1100_q10/

qiime tools import \
--input-path ./seqtable_mangrove_arch_ONT.fa \
--type 'FeatureData[Sequence]' \
--output-path ./rep-seqs_mangrove_arch_ONT.qza

In [ ]:
biom convert -i ./16S_ONT_1400_1600_q10/seqtable_mangrove_bact_ONT.txt -o ./16S_ONT_1400_1600_q10/table_mangrove_bact_ONT_final.biom --table-type="OTU table" --to-hdf5
biom convert -i ./arch_ONT_900_1100_q10/seqtable_mangrove_arch_ONT.txt -o ./arch_ONT_900_1100_q10/table_mangrove_arch_ONT_final.biom --table-type="OTU table" --to-hdf5

qiime tools import \
--input-path ./16S_ONT_1400_1600_q10/table_mangrove_bact_ONT_final.biom \
--type 'FeatureTable[Frequency]' \
--output-path ./16S_ONT_1400_1600_q10/table_mangrove_bact_ONT_final.qza

qiime tools import \
--input-path ./arch_ONT_900_1100_q10//table_mangrove_arch_ONT_final.biom \
--type 'FeatureTable[Frequency]' \
--output-path ./arch_ONT_900_1100_q10//table_mangrove_arch_ONT_final.qza

In [ ]:
# Make 97% consensus sequences (OTU) from ASVs

In [ ]:
##############
# for bacteria
cd ./16S_ONT_1400_1600_q10/
qiime vsearch cluster-features-de-novo \
  --i-table table_mangrove_bact_ONT_final.qza \
  --i-sequences rep-seqs_mangrove_bact_ONT.qza \
  --p-perc-identity 0.97 \
  --p-threads 32 \
  --o-clustered-table table-97_mangrove_bact_ONT_final.qza \
  --o-clustered-sequences rep-seqs-97_mangrove_bact_ONT_final.qza

##############
# for archaea
cd ./arch_ONT_900_1100_q10/
qiime vsearch cluster-features-de-novo \
  --i-table table_mangrove_arch_ONT_final.qza \
  --i-sequences rep-seqs_mangrove_arch_ONT.qza \
  --p-perc-identity 0.97 \
  --p-threads 32 \
  --o-clustered-table table-97_mangrove_arch_ONT_final.qza \
  --o-clustered-sequences rep-seqs-97_mangrove_arch_ONT_final.qza

In [ ]:
# Export OTU tables

In [ ]:
##############
# for bacteria
cd ./16S_ONT_1400_1600_q10/
qiime tools export \
    --input-path table-97_mangrove_bact_ONT_final.qza \
    --output-path ./
    
biom convert \
    -i ./feature-table.biom \
    -o ./feature-table-97_mangrove_bact_ONT_final.tsv --to-tsv

##############
# for archaea
cd ./arch_ONT_900_1100_q10/
qiime tools export \
    --input-path table-97_mangrove_arch_ONT_final.qza \
    --output-path ./
    
biom convert \
    -i ./feature-table.biom \
    -o ./feature-table-97_mangrove_arch_ONT_final.tsv --to-tsv


# open the .tsv file with LibreOffice or Excel, select abundances, then Edition > Find and replace,
# Seek for ^[0-9] and replace it by & (tick "regular expressions")

In [ ]:
# Taxonomic assignation of 16SV4V5 sequences with the database SILVA 138.1-ssu-nr99 fitted to the approriate markers
# classifier must have been prepared before, see https://docs.qiime2.org/2023.5/tutorials/feature-classifier/
##############
# for bacteria
cd ./16S_ONT_1400_1600_q10/
qiime feature-classifier classify-sklearn \
   --i-reads ./rep-seqs-97_mangrove_bact_ONT_final.qza\
   --i-classifier ./silva-138.1-ssu-nr99-full16S-classifier.qza \
   --p-n-jobs -1 \
   --output-dir ./Assignation

# export in .biom format
qiime tools export \
    --input-path ./OTU-bact-taxo/classification.qza \
    --output-path ./OTU-bact-taxo

##############
# for archaea
cd ./arch_ONT_900_1100_q10/
qiime feature-classifier classify-sklearn \
   --i-reads ./rep-seqs-97_mangrove_arch_ONT_final.qza\
   --i-classifier ./silva-138.1-ssu-nr99-full16S-classifier.qza \
   --p-n-jobs -1 \
   --output-dir ./Assignation

# export in .biom format
qiime tools export \
    --input-path ./OTU-arch-taxo/classification.qza \
    --output-path ./OTU-arch-taxo

In [ ]:
# Then have been created for each domain :
# - an OTU table at 3% (sequences and read abundances after quality filters) [feature-table-97_mangrove_bact_ONT_final.tsv]
# - a table with OTU taxonomical assignations [taxonomy.tsv]

# User shall then merge them, strictly keeping the order of the lines.